In [ ]:
%load_ext nb_black
%load_ext autoreload
%autoreload 2

import os

print(os.getcwd())


def update_working_directory():
    from pathlib import Path

    p = Path(os.getcwd()).parents[0]
    os.chdir(p)
    print(p)


update_working_directory()

In [ ]:
import numpy as np
import pandas as pd

import src.data.get_dataset as get_dataset

In [ ]:
current_vocab = get_dataset.get_vocab(
    "data/official/german_english.csv", list_columns="all"
)
current_vocab

In [ ]:
word_files = [
    "wortschatzliste_einfach-gut_a1-1_englisch_clean",
    "wortschatzliste_einfach-gut_a1-2_englisch_clean",
    "5points",
    "4points",
    "3points",
    # "2points",
    # "1points",
    # "0points",
    # "Minus1points",
    # "Minus2points",
    # "Minus3points",
    # "Minus4points",
    # "Minus5points",
    # "Minus6points",
    # "Minus7points",
    # "Minus8points",
    # "Minus9points",
    # "Minus10points",
]

In [ ]:
for word_file in word_files:

    new_vocab = pd.read_csv(f"data/raw/new_vocabulary/{word_file}.csv")

    new_vocab = new_vocab[
        (~new_vocab["English"].isin(current_vocab["english"]))
        & (~new_vocab["German"].isin(current_vocab["german"]))
    ]

    if len(new_vocab) == 0:
        print(f"no new word to add for {word_file}")
        continue

    new_vocab = new_vocab.sample(min(100000, len(new_vocab)))

    cleaned_new_vocab = pd.DataFrame()

    if len(current_vocab) == 0:
        last_id = 0
    else:
        last_id = max(current_vocab["id_vocab"])

    cleaned_new_vocab["id_vocab"] = list(
        range(last_id + 1, last_id + 1 + len(new_vocab))
    )
    cleaned_new_vocab["german"] = new_vocab["German"].values.tolist()
    cleaned_new_vocab["english"] = new_vocab["English"].values.tolist()
    cleaned_new_vocab["score_german_english"] = 0
    cleaned_new_vocab["score_english_german"] = 0
    cleaned_new_vocab["try_session_german_english"] = False
    cleaned_new_vocab["try_session_english_german"] = False

    current_vocab = current_vocab.append(cleaned_new_vocab)

    print(f"{len(new_vocab)} new words were added from {word_file}")

In [ ]:
current_vocab

In [ ]:
current_vocab.to_csv("data/official/german_english.csv", index=False)